In [1]:
date = 20220930    #current date

import numpy as np
import pandas as pd
import pandasql
import sqlite3

pd.set_option("display.max_columns", None) 
pd.set_option("display.max_colwidth", 1000) #huruf dlm column
pd.set_option("display.max_rows", 100)
pd.set_option("display.precision", 2) #2 titik perpuluhan


LN = pd.read_csv(r"T:\MIB Risk Management\Credit Risk Analytics - Historical\04_Infocube\01_Loans\\"+str(date)+"\infocube_01_loan"+str(date)+".txt", sep = ",", header = 0, low_memory = False)
LN.columns = LN.columns.str.replace(" ", "_")
LN.M_Account_No = LN.M_Account_No.astype(str)

OD  = pd.read_csv(r"T:\MIB Risk Management\Credit Risk Analytics - Historical\04_Infocube\02_OD\\"+str(date)+"\\infocube_02_OD"+str(date)+".txt", sep = ",", header = 0, low_memory = False)
OD.columns = OD.columns.str.replace(" ", "_")
OD.Account_No = OD.Account_No.astype(str)

INV  = pd.read_csv(r"T:\MIB Risk Management\Credit Risk Analytics - Historical\04_Infocube\03_Investment\\"+str(date)+"\infocube_03_inv"+str(date)+".txt", sep = ",", header = 0, low_memory = False)
INV.columns = INV.columns.str.replace(" ", "_")
INV.M_Account_No = INV.M_Account_No.astype(str)

TRADE  = pd.read_csv(r"T:\MIB Risk Management\Credit Risk Analytics - Historical\04_Infocube\04_Trade\\"+str(date)+"\infocube_04_Trade"+str(date)+".txt", sep = ",", header = 0, low_memory = False)
TRADE.columns = TRADE.columns.str.replace(" ", "_")
TRADE.Account_No = TRADE.Account_No.astype(str)

FCL  = pd.read_csv(r"T:\MIB Risk Management\Credit Risk Analytics - Historical\04_Infocube\05_FCL\\"+str(date)+"\infocube_05_FCL"+str(date)+".txt", sep = ",", header = 0, low_memory = False)
FCL.columns = FCL.columns.str.replace(" ", "_")
FCL.M_Account_No = FCL.M_Account_No.astype(str)

HP  = pd.read_csv(r"T:\MIB Risk Management\Credit Risk Analytics - Historical\04_Infocube\06_AITAB\\"+str(date)+"\infocube_06_aitab"+str(date)+".txt", sep = ",", header = 0, low_memory = False)
HP.columns = HP.columns.str.replace(" ", "_")
HP.ACCOUNT_NO = HP.ACCOUNT_NO.astype(str)
HP['M_Account_No'] = HP.ACCOUNT_NO.str[3:]
#HP['ACCOUNT_NO'] = HP.ACCOUNT_NO.str[3:] xyah sbb x sme

#RDMS = pd.read_csv(r"T:\MIB Risk Management\Credit Risk Analytics - Historical\04_Infocube\99_Data_Source\01_Datawarehouse\01_Credit_Risk_Reporting\04_MIB_Rdms\\"+str(date)+"\RDMS_"+str(date)+".txt", sep = ",", header = 0, error_bad_lines = False)
#RDMS.columns = RDMS.columns.str.replace(" ", "_")
#RDMS.Account_No = RDMS.Account_No.astype(str)

CC  = pd.read_csv(r"T:\MIB Risk Management\Credit Risk Analytics - Historical\04_Infocube\12_CC\\"+str(date)+"\infocube_12_CC"+str(date)+".txt", sep = ",", header = 0, low_memory = False)
CC.columns = CC.columns.str.replace(" ", "_")
CC.M_Account_No = CC.M_Account_No.astype(str)

TOTAL_IMPAIRED_FINAL  = pd.read_csv(r'T:\MIB Risk Management\Credit Risk Analytics - Historical\02_Data_Source\20_CRP\6.0 Impaired Loan\\'+str(date)[:4]+'\\'+str(date)[:6]+'\working\Total_Impaired_Final_'+str(date)+'.txt',sep = ",", header = 0, low_memory = False)
TOTAL_IMPAIRED_FINAL.columns = TOTAL_IMPAIRED_FINAL.columns.str.replace(" ", "_")
TOTAL_IMPAIRED_FINAL.V_ORIGINAL_ACCOUNT_NUMBER = TOTAL_IMPAIRED_FINAL.V_ORIGINAL_ACCOUNT_NUMBER.astype(str)
TOTAL_IMPAIRED_FINAL.columns = TOTAL_IMPAIRED_FINAL.columns.str.replace("(", "_").str.replace(")", "_").str.replace(".", "_").str.replace("-", "_").str.replace("\n", "_").str.replace(" ", "_")

RPSIA_ACCTLEVEL2 = pd.read_csv(r'T:\MIB Risk Management\Credit Risk Analytics - Historical\02_Data_Source\20_CRP\10.0 RPSIA\\'+str(date)[:4]+'\\IF_RPSIA_ACCTLEVEL2_'+str(date)+'.txt',sep = ",", header = 0, low_memory = False)
RPSIA_ACCTLEVEL2.columns = RPSIA_ACCTLEVEL2.columns.str.replace(" ", "_")

In [4]:
LOAN1 = LN

LOAN1.columns = LOAN1.columns.str.upper()

LOAN1 = LOAN1.iloc[np.where((LOAN1.M_ACCT_STATUS_CD==1)&(LOAN1.M_FULL_WRITEOFF_FLG=='N'))]

LOAN1['DATA_SOURCE'] = '01. LOAN'
LOAN1['SECURITY_TYPE_CD'] = -9999

LOAN1 = LOAN1[['DATA_SOURCE'\
,'AS_OF_DATE'\
,'M_ACCOUNT_NO'\
,'LEVEL_6'\
,'M_SUB_MARKET_SEGMENT_CD'\
,'M_PRODUCT_HIERARCHY_CD'\
,'M_SOURCE_PROD_TYPE_CD'\
,'SOURCE_PROD_TYPE_DSC'\
,'M_MTHS_INSTALLMENT_IN_ARREARS'\
,'SECURITY_TYPE_CD'\
,'M_FINANCING_CONCEPT'\
,'M_BNM_BALANCE_SUM']]

OD1 = OD

OD1.columns = OD1.columns.str.upper()

OD1 = OD1.iloc[np.where((OD1.ACCT_STATUS_CD!=3)&(OD1.FULL_WRITEOFF_FLG=='N')&(OD1.COMMON_COA_ID==12000))]

OD1['DATA_SOURCE'] = '02. OD'

OD1 = OD1[['DATA_SOURCE'\
,'AS_OF_DATE'\
,'ACCOUNT_NO'\
,'LEVEL_6'\
,'M_SUB_MARKET_SEGMENT_CD'\
,'PRODUCT_HIERARCHY_CD'\
,'SOURCE_PROD_TYPE_CD'\
,'SOURCE_PROD_TYPE_DSC'\
,'M_MTHS_IN_ARREARS'\
,'SECURITY_TYPE_CD'\
,'M_FINANCING_CONCEPT'\
,'M_BNM_BALANCE']]


STRC1 = INV

STRC1.columns = STRC1.columns.str.upper()

STRC1 = STRC1.iloc[np.where(STRC1.LEVEL_6=='Short Term Revolving Credit')]

STRC1['DATA_SOURCE'] = '03. STRC'
STRC1['M_MTHS_INSTALLMENT_IN_ARREARS'] = -9999
STRC1['SECURITY_TYPE_CD'] = -9999
STRC1['M_FINANCING_CONCEPT'] = -9999

STRC1 = STRC1[['DATA_SOURCE'
,'AS_OF_DATE'
,'M_ACCOUNT_NO'
,'LEVEL_6'
,'M_SUB_MARKET_SEGMENT_CD'
,'M_PRODUCT_HIERARCHY_CD'
,'M_SOURCE_PROD_TYPE_CD'
,'SOURCE_PROD_TYPE_DSC'
,'M_MTHS_INSTALLMENT_IN_ARREARS'
,'SECURITY_TYPE_CD'	
,'M_FINANCING_CONCEPT'
,'M_BNM_BALANCE']]


INV1 = INV

INV1.columns = INV1.columns.str.upper()

INV1 = INV1.iloc[np.where(INV1.LEVEL_6.isin(['Foreign Currency Loan','Owned BA Discounted']))]

INV1['DATA_SOURCE'] = '04. INV'
INV1['M_MTHS_INSTALLMENT_IN_ARREARS'] = -9999
INV1['SECURITY_TYPE_CD'] = -9999
INV1['M_FINANCING_CONCEPT'] = -9999

INV1 = INV1[['DATA_SOURCE'\
,'AS_OF_DATE'\
,'M_ACCOUNT_NO'\
,'LEVEL_6'\
,'M_SUB_MARKET_SEGMENT_CD'\
,'M_PRODUCT_HIERARCHY_CD'\
,'M_SOURCE_PROD_TYPE_CD'\
,'SOURCE_PROD_TYPE_DSC'\
,'M_MTHS_INSTALLMENT_IN_ARREARS'\
,'SECURITY_TYPE_CD'\
,'M_FINANCING_CONCEPT'\
,'M_BNM_BALANCE']]

TRADE1 = TRADE

TRADE1.columns = TRADE1.columns.str.upper()

TRADE1 = TRADE1.iloc[np.where(~(TRADE1.PDT_DESCRIPTION.isin(['OWN IAB ACCEPT DISCOUNT', 'ISLAMIC - ONSHORE FOREIGN CURRENCY LOAN']))&(TRADE1.FUNDED_FLG=='Y'))]

TRADE1['DATA_SOURCE'] = '05. TRADE'
TRADE1['M_MTHS_INSTALLMENT_IN_ARREARS'] = -9999
TRADE1['SECURITY_TYPE_CD'] = -9999

TRADE1 = TRADE1[['DATA_SOURCE'\
,'AS_OF_DATE'\
,'ACCOUNT_NO'\
,'PDT_DESCRIPTION'\
,'M_SUB_MARKET_SEGMENT_CD'\
,'PRODUCT_HIERARCHY_CD'\
,'SOURCE_PROD_TYPE_CD'\
,'SOURCE_PROD_TYPE_DSC'\
,'M_MTHS_INSTALLMENT_IN_ARREARS'\
,'SECURITY_TYPE_CD'\
,'M_FINANCING_CONCEPT'\
,'M_BNM_BALANCE']].rename(columns={'PDT_DESCRIPTION':'LEVEL_6'})


FCL1 = FCL

FCL1.columns = FCL1.columns.str.upper()

FCL1 = FCL1.iloc[np.where(FCL1.RECORD_COUNT==0)]

FCL1['DATA_SOURCE'] = '06. FCL'
FCL1['SECURITY_TYPE_CD'] = -9999

FCL1 = FCL1[['DATA_SOURCE'\
,'AS_OF_DATE'\
,'M_ACCOUNT_NO'\
,'LEVEL_6'\
,'M_SUB_MARKET_SEGMENT_CD'\
,'M_PRODUCT_HIERARCHY_CD'\
,'M_SOURCE_PROD_TYPE_CD'\
,'SOURCE_PROD_TYPE_DSC'\
,'M_MTHS_INSTALLMENT_IN_ARREARS'\
,'SECURITY_TYPE_CD'\
,'M_FINANCING_CONCEPT'\
,'M_BNM_BALANCE']] 

HP.columns = HP.columns.str.upper()

HP1 = HP.fillna(0).groupby(['ACCOUNT_NO'])[['BNM_BALANCE']].sum().reset_index()

HP1 = HP1.merge(HP[['AS_OF_DATE'
,'ACCOUNT_NO'
,'LEVEL_6'
,'M_SUB_MARKET_SEGMENT_CD'
,'PRODUCT_HIERARCHY_CD'
,'SOURCE_PROD_TYPE_CD'
,'SOURCE_PROD_TYPE_DSC'
,'MTHS_INSTALLMENT_IN_ARREARS'
,'FINANCING_CONCEPT']].drop_duplicates('ACCOUNT_NO'),on='ACCOUNT_NO',how='left')

HP1['DATA_SOURCE'] = '07. HP'
HP1['SECURITY_TYPE_CD'] = -9999

HP1 = HP1[['DATA_SOURCE'\
,'AS_OF_DATE'\
,'ACCOUNT_NO'\
,'LEVEL_6'\
,'M_SUB_MARKET_SEGMENT_CD'\
,'PRODUCT_HIERARCHY_CD'\
,'SOURCE_PROD_TYPE_CD'\
,'SOURCE_PROD_TYPE_DSC'\
,'MTHS_INSTALLMENT_IN_ARREARS'\
,'SECURITY_TYPE_CD'\
,'FINANCING_CONCEPT'\
,'BNM_BALANCE']]

CC1 = CC

CC1.columns = CC1.columns.str.upper()

CC1['DATA_SOURCE'] = '08. CC'
CC1['SECURITY_TYPE_CD'] = -9999
CC1['M_FINANCING_CONCEPT'] = -9999

CC1=CC1[['DATA_SOURCE'\
,'AS_OF_DATE'\
,'M_ACCOUNT_NO'\
,'LEVEL_6'\
,'M_SUB_MARKET_SEGMENT_CD'\
,'M_PRODUCT_HIERARCHY_CD'\
,'M_SOURCE_PROD_TYPE_CD'\
,'SOURCE_PROD_TYPE_DSC'\
,'M_MTHS_IN_ARREARS'\
,'SECURITY_TYPE_CD'\
,'M_FINANCING_CONCEPT'\
,'M_BNM_BALANCE2']]

LOAN1.columns=OD1.columns=STRC1.columns=INV1.columns=TRADE1.columns=FCL1.columns=HP1.columns=CC1.columns

frames = [LOAN1 , OD1 , STRC1 , INV1 , TRADE1 , FCL1 , HP1 , CC1] 
A001 = pd.concat(frames)

D:\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
D:\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
D:\anaconda3\lib\site-packages\ipykernel_launcher.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gu

In [17]:
LOAN1.shape, OD1.shape, STRC1.shape, INV1.shape, TRADE1.shape, FCL1.shape, HP1.shape, CC1.shape

((759808, 12),
 (17219, 12),
 (696, 12),
 (9964, 12),
 (6575, 12),
 (224, 12),
 (988994, 12),
 (419267, 12))

In [5]:
TOTAL_IMPAIRED_FINAL.loc[TOTAL_IMPAIRED_FINAL.PROD_LV1=='On Balance Sheet','IMPAIRED_FLAG']='Y'

A002 = A001.merge(TOTAL_IMPAIRED_FINAL[['V_ORIGINAL_ACCOUNT_NUMBER','IMPAIRED_FLAG','F_NCIF_CUST','F_NCIF_ACCT','OUTSTANDING_AFTER_EIR_UWI','GROSS_LOAN']].rename(columns={'V_ORIGINAL_ACCOUNT_NUMBER':'M_ACCOUNT_NO'}),on='M_ACCOUNT_NO',how='left')

A003 = A002.fillna(0).groupby(['DATA_SOURCE'\
,'AS_OF_DATE'\
,'LEVEL_6'\
,'M_SUB_MARKET_SEGMENT_CD'\
,'M_PRODUCT_HIERARCHY_CD'\
,'M_SOURCE_PROD_TYPE_CD'\
,'SOURCE_PROD_TYPE_DSC'\
,'M_MTHS_IN_ARREARS'\
,'IMPAIRED_FLAG'\
,'F_NCIF_CUST'\
,'F_NCIF_ACCT'\
,'SECURITY_TYPE_CD'\
,'M_FINANCING_CONCEPT'])[['M_BNM_BALANCE2','OUTSTANDING_AFTER_EIR_UWI','GROSS_LOAN']].sum().reset_index()

A003.fillna(0, inplace=True)
A003.DATA_SOURCE = A003.DATA_SOURCE.astype(str)
A003.LEVEL_6 = A003.LEVEL_6.astype(str)
A003.loc[A003.M_SUB_MARKET_SEGMENT_CD.isin(['None','NAN']), 'M_SUB_MARKET_SEGMENT_CD'] = 0
A003.M_SUB_MARKET_SEGMENT_CD = A003.M_SUB_MARKET_SEGMENT_CD.astype(float)
A003.M_PRODUCT_HIERARCHY_CD = A003.M_PRODUCT_HIERARCHY_CD.astype(str)
A003.SECURITY_TYPE_CD = A003.SECURITY_TYPE_CD.astype(int)

def PRODUCT_GROUP_APC(DATA_SOURCE,Level_6,M_Sub_Market_Segment_Cd,M_Product_Hierarchy_Cd,Security_Type_Cd):
    #NON-RETAIL FINANCING
    if( ((DATA_SOURCE!='07. HP')&(DATA_SOURCE!='08. CC'))&((DATA_SOURCE == '01. LOAN')|(DATA_SOURCE == '02. OD')|(DATA_SOURCE == '03. STRC')|(DATA_SOURCE == '04. INV')|(DATA_SOURCE == '05. TRADE')|(DATA_SOURCE == '06. FCL'))&((Level_6!='Salary')&(Level_6!='SMF'))):
        if((M_Sub_Market_Segment_Cd == 5)|(M_Sub_Market_Segment_Cd == 6)|(M_Sub_Market_Segment_Cd == 7)|(M_Sub_Market_Segment_Cd == 8)):
            return '01. Corporate Banking'
        elif(M_Sub_Market_Segment_Cd == 2):
            return '02. Business Banking'
        elif(M_Sub_Market_Segment_Cd == 1):
            return '03. RSME Banking'
    #RETAIL FINANCING
    #MORTGAGE FINANCING
    if(((DATA_SOURCE == '01. LOAN')|(DATA_SOURCE == '02. OD')|(DATA_SOURCE == '06. FCL'))&((M_Sub_Market_Segment_Cd == 0)|(M_Sub_Market_Segment_Cd == 3)|(M_Sub_Market_Segment_Cd == 4))):
        if(Level_6 == 'Housing'):
            return '04. Home Financing'
        elif(Level_6 == 'Shophouse'):
            return '05. Shophouse'
        elif(Level_6 == 'Other Mortgage Loan'):
            return '06. Other Mortgage'
    #HIRE PURCHASE
    if(DATA_SOURCE == '07. HP'):
        if(M_Product_Hierarchy_Cd == 'N2'):
            return '07. Hire Purchase-N2'
        elif(M_Product_Hierarchy_Cd == 'N4'):
            return '07. Hire Purchase-N4'
        else:
            return '07. Hire Purchase-OTH'
    #PRODUCT EXCLUDE FROM PERSONAL FINANCING ONLY WO CARDS
    if((DATA_SOURCE == '01. LOAN')&((M_Product_Hierarchy_Cd == '711')|(M_Product_Hierarchy_Cd == '714'))):
        return '08. Credit Cards-PF'
    if(DATA_SOURCE == '08. CC'):
        return '08. Credit Cards'
    #UNIT TRUST
    if((DATA_SOURCE == '01. LOAN')&((M_Sub_Market_Segment_Cd == 0)|(M_Sub_Market_Segment_Cd == 3)|(M_Sub_Market_Segment_Cd == 4))):
        if((Level_6 == 'Unit Trust')|(Level_6 == 'Unit Trust-ASB')|(Level_6 == 'Unit Trust-ASB2')):
            return '09. Unit Trust'
    #SALARY
    if(Level_6=='Salary'):
        return '11. Salary'
    #OTHER RETAIL FINANCING
    if((DATA_SOURCE == '01. LOAN')&((M_Sub_Market_Segment_Cd == 0)|(M_Sub_Market_Segment_Cd == 3)|(M_Sub_Market_Segment_Cd == 4))):
        if(((Level_6!='SMF')&(Level_6!='Unit Trust')&(Level_6!='Unit Trust-ASB')&(Level_6!='Unit Trust-ASB2')&\
            (Level_6!='Housing')&(Level_6!='Shophouse')&(Level_6!='Other Mortgage Loan')&(Level_6!='Salary'))&\
           (DATA_SOURCE == '01. LOAN')&((M_Product_Hierarchy_Cd!='711')&(M_Product_Hierarchy_Cd!='714')&\
                                        (M_Product_Hierarchy_Cd!='HZK01')&(M_Product_Hierarchy_Cd!='HZK02')&\
                                        (M_Product_Hierarchy_Cd!='HZK03')&(M_Product_Hierarchy_Cd!='HZK04'))):
            if((M_Product_Hierarchy_Cd=='H28')|(M_Product_Hierarchy_Cd=='J9')):
                return '12. OCF SECURED-APC'
            elif((M_Product_Hierarchy_Cd=='H61')|(M_Product_Hierarchy_Cd=='H62')|(M_Product_Hierarchy_Cd=='W3')|(M_Product_Hierarchy_Cd=='W16')):
                return '12. OCF UNSECURED-APC'
            else:
                return '12. Other Retail Financing'
    #CASHLINE
    if((DATA_SOURCE=='02. OD')&((M_Sub_Market_Segment_Cd == 0)|(M_Sub_Market_Segment_Cd == 3)|(M_Sub_Market_Segment_Cd == 4))&\
       ((Level_6!='SMF')&(Level_6!='Housing')&(Level_6!='Shophouse')&(Level_6!='Other Mortgage Loan'))):
        if((Security_Type_Cd== 57)|(Security_Type_Cd==42)):
            return '13. Cashline-APC'
        else:
            return '13. Cashline-OTH'
    #SMF
    if(Level_6=='SMF'):
        if((M_Sub_Market_Segment_Cd == 0)|(M_Sub_Market_Segment_Cd == 3)|(M_Sub_Market_Segment_Cd == 4)):
            return '14. SMF-APC'
        else:
            return '14. SMF-OTH'
    #RTO
    if((DATA_SOURCE=='01. LOAN')&(Level_6=='RTO')):
        return '15. CB-RTO'
A003['PRODUCT_GROUP_APC'] = A003.apply(lambda x: PRODUCT_GROUP_APC(x['DATA_SOURCE'], x['LEVEL_6'], x['M_SUB_MARKET_SEGMENT_CD'],x['M_PRODUCT_HIERARCHY_CD'],x['SECURITY_TYPE_CD']), axis=1)


In [ ]:
A003.loc[A003.PRODUCT_GROUP_APC.isin(['04. Home Financing','05. Shophouse','06. Other Mortgage']),'PRODUCT_GROUP_APC2'] = '01. MORTGAGE'
A003.loc[A003.PRODUCT_GROUP_APC.isin(['07. Hire Purchase-N2']),'PRODUCT_GROUP_APC2'] = '01. MORTGAGE'
A003.loc[A003.PRODUCT_GROUP_APC.isin(['07. Hire Purchase-N4']),'PRODUCT_GROUP_APC2'] = '01. MORTGAGE'
A003.loc[A003.PRODUCT_GROUP_APC.isin(['08. Credit Cards']),'PRODUCT_GROUP_APC2'] = '01. MORTGAGE'
A003.loc[A003.PRODUCT_GROUP_APC.isin(['08. Credit Cards-PF']),'PRODUCT_GROUP_APC2'] = '01. MORTGAGE'
A003.loc[A003.PRODUCT_GROUP_APC.isin(['09. Unit Trust']),'PRODUCT_GROUP_APC2'] = '01. MORTGAGE'
A003.loc[A003.PRODUCT_GROUP_APC.isin(['12. OCF SECURED-APC','12. OCF UNSECURED-APC','13. Cashline-APC']),'PRODUCT_GROUP_APC2'] = '01. MORTGAGE'
A003.loc[A003.PRODUCT_GROUP_APC.isin(['04. Home Financing','05. Shophouse','06. Other Mortgage']),'PRODUCT_GROUP_APC2'] = '01. MORTGAGE'
A003.loc[A003.PRODUCT_GROUP_APC.isin(['04. Home Financing','05. Shophouse','06. Other Mortgage']),'PRODUCT_GROUP_APC2'] = '01. MORTGAGE'
A003.loc[A003.PRODUCT_GROUP_APC.isin(['04. Home Financing','05. Shophouse','06. Other Mortgage']),'PRODUCT_GROUP_APC2'] = '01. MORTGAGE'
A003.loc[A003.PRODUCT_GROUP_APC.isin(['04. Home Financing','05. Shophouse','06. Other Mortgage']),'PRODUCT_GROUP_APC2'] = '01. MORTGAGE'

CASE 
WHEN PRODUCT_GROUP_APC IN ('04. Home Financing','05. Shophouse','06. Other Mortgage') THEN '01. MORTGAGE'
WHEN PRODUCT_GROUP_APC = '07. Hire Purchase-N2' THEN '02. HIRE PURCHASE-N2'
WHEN PRODUCT_GROUP_APC = '07. Hire Purchase-N4' THEN '03. HIRE PURCHASE-N4'
WHEN PRODUCT_GROUP_APC = '08. Credit Cards' THEN '04. CREDIT CARDS'
WHEN PRODUCT_GROUP_APC = '08. Credit Cards-PF' THEN '05. CREDIT CARDS-PF'
WHEN PRODUCT_GROUP_APC = '09. Unit Trust' THEN '06. UNIT TRUST'
WHEN PRODUCT_GROUP_APC IN ('12. OCF SECURED-APC','12. OCF UNSECURED-APC','13. Cashline-APC') THEN '07. OCF' /*remove '14. SMF-APC'*/
WHEN PRODUCT_GROUP_APC = '11. Salary' THEN '08. SALARY'
WHEN PRODUCT_GROUP_APC = '03. RSME Banking' THEN '09. RSME' 
WHEN DATA_SOURCE='01. LOAN' AND Level_6 ='RTO' THEN '15. RTO' 
WHEN LEVEL_6 = 'SMF' THEN '16. SMF'
END AS PRODUCT_GROUP_APC2 ,

In [19]:
#==============================================================================

eonn = sqlite3.connect(r'T:\MIB Risk Management\Credit Risk Analytics - Historical\01_Reporting\01_Monthly_Reporting\13_Adverse Portfolio\\'+str(date)[:4]+'\\'+str(date)+'\\working\APC.db')
e = eonn.cursor()

e.execute("CREATE TABLE IF NOT EXISTS A004 ({})".format(','.join(A003.columns)))

for row in A003.iterrows():
    sql = "INSERT INTO A004({}) VALUES ({})".format(','.join(A003.columns), ','.join(['?']*len(A003.columns)))
    e.execute(sql, tuple(row[1]))

eonn.commit()

In [20]:
e.execute("""
CREATE TABLE A005 AS SELECT

A.*,
CASE 
WHEN PRODUCT_GROUP_APC IN ('04. Home Financing','05. Shophouse','06. Other Mortgage') THEN '01. MORTGAGE'
WHEN PRODUCT_GROUP_APC = '07. Hire Purchase-N2' THEN '02. HIRE PURCHASE-N2'
WHEN PRODUCT_GROUP_APC = '07. Hire Purchase-N4' THEN '03. HIRE PURCHASE-N4'
WHEN PRODUCT_GROUP_APC = '08. Credit Cards' THEN '04. CREDIT CARDS'
WHEN PRODUCT_GROUP_APC = '08. Credit Cards-PF' THEN '05. CREDIT CARDS-PF'
WHEN PRODUCT_GROUP_APC = '09. Unit Trust' THEN '06. UNIT TRUST'
WHEN PRODUCT_GROUP_APC IN ('12. OCF SECURED-APC','12. OCF UNSECURED-APC','13. Cashline-APC') THEN '07. OCF' /*remove '14. SMF-APC'*/
WHEN PRODUCT_GROUP_APC = '11. Salary' THEN '08. SALARY'
WHEN PRODUCT_GROUP_APC = '03. RSME Banking' THEN '09. RSME' 
WHEN DATA_SOURCE='01. LOAN' AND Level_6 ='RTO' THEN '15. RTO' 
WHEN LEVEL_6 = 'SMF' THEN '16. SMF'
END AS PRODUCT_GROUP_APC2 ,

CASE WHEN PRODUCT_GROUP_APC NOT IN ('08. Credit Cards','08. Credit Cards-PF') AND F_NCIF_CUST = 'Y' THEN OUTSTANDING_AFTER_EIR_UWI  ELSE 0
END AS OUTSTANDING_AFTER_EIR_UWI_NCIF,

CASE WHEN PRODUCT_GROUP_APC NOT IN ('08. Credit Cards','08. Credit Cards-PF') AND F_NCIF_ACCT = 'Y' THEN OUTSTANDING_AFTER_EIR_UWI  ELSE 0 
END AS OS_AFTER_EIR_UWI_NCIF_ACCT,

/*#add 3/11/2022*/
CASE WHEN PRODUCT_GROUP_APC NOT IN ('08. Credit Cards','08. Credit Cards-PF') AND F_NCIF_CUST = 'Y' THEN GROSS_LOAN  ELSE 0
END AS GROSS_LOAN_NCIF,

CASE WHEN PRODUCT_GROUP_APC NOT IN ('08. Credit Cards','08. Credit Cards-PF') AND F_NCIF_ACCT = 'Y' THEN GROSS_LOAN  ELSE 0 
END AS GROSS_LOAN_NCIF_ACCT,

CASE 
WHEN PRODUCT_GROUP_APC NOT IN ('08. Credit Cards','08. Credit Cards-PF') AND IMPAIRED_FLAG <> 'Y' AND M_MTHS_IN_ARREARS IN (2,3) THEN M_BNM_BALANCE2
WHEN PRODUCT_GROUP_APC IN ('08. Credit Cards','08. Credit Cards-PF') AND IMPAIRED_FLAG <> 'Y' AND M_MTHS_IN_ARREARS = 1 THEN M_BNM_BALANCE2 ELSE 0
END AS M_BNM_BALANCE2_MIA23,

CASE 
WHEN /*PRODUCT_GROUP_APC NOT IN ('08. Credit Cards','08. Credit Cards-PF') AND*/ IMPAIRED_FLAG <> 'Y' AND M_MTHS_IN_ARREARS = 2 THEN M_BNM_BALANCE2 ELSE 0
END AS M_BNM_BALANCE2_MIA2,

CASE 
WHEN PRODUCT_GROUP_APC NOT IN ('08. Credit Cards','08. Credit Cards-PF') AND IMPAIRED_FLAG <> 'Y' AND M_MTHS_IN_ARREARS = 3 THEN M_BNM_BALANCE2 ELSE 0
END AS M_BNM_BALANCE2_MIA3

FROM A004 A;
""")

eonn.commit()

#APC.FINAL_APC_&RDATE = A005;


#/*Reports Layout*/

e.execute("""
CREATE TABLE SUMM_001 AS SELECT
PRODUCT_GROUP_APC2 AS PRODUCT_GROUP
,SUM(M_BNM_BALANCE2) AS OSBAL
,SUM(OUTSTANDING_AFTER_EIR_UWI) AS OSBAL_IF
,SUM(OUTSTANDING_AFTER_EIR_UWI_NCIF) AS OSBAL_NCIF
,SUM(OS_AFTER_EIR_UWI_NCIF_ACCT) AS OSBAL_NCIF_ACCT

,SUM(GROSS_LOAN) AS GROSS_IF
,SUM(GROSS_LOAN_NCIF) AS GROSS_NCIF
,SUM(GROSS_LOAN_NCIF_ACCT) AS GROSS_NCIF_ACCT

,SUM(M_BNM_BALANCE2_MIA23) AS OSBAL_MIA23
,SUM(M_BNM_BALANCE2_MIA2) AS OSBAL_MIA2
,SUM(M_BNM_BALANCE2_MIA3) AS OSBAL_MIA3
, SUM(OUTSTANDING_AFTER_EIR_UWI)/SUM(M_BNM_BALANCE2) AS IF_RATIO
, SUM(OUTSTANDING_AFTER_EIR_UWI_NCIF)/SUM(M_BNM_BALANCE2) AS NCIF_RATIO
, SUM(OS_AFTER_EIR_UWI_NCIF_ACCT)/SUM(M_BNM_BALANCE2) AS NCIF_ACCT_RATIO

, SUM(GROSS_LOAN)/SUM(M_BNM_BALANCE2) AS GROSS_IF_RATIO
, SUM(GROSS_LOAN_NCIF)/SUM(M_BNM_BALANCE2) AS GROSS_NCIF_RATIO
, SUM(GROSS_LOAN_NCIF_ACCT)/SUM(M_BNM_BALANCE2) AS GROSS_NCIF_ACCT_RATIO

, SUM(M_BNM_BALANCE2_MIA23)/SUM(M_BNM_BALANCE2) AS MIA23_RATIO
, SUM(M_BNM_BALANCE2_MIA2)/SUM(M_BNM_BALANCE2) AS MIA2_RATIO
, SUM(M_BNM_BALANCE2_MIA3)/SUM(M_BNM_BALANCE2) AS MIA3_RATIO
FROM A005
WHERE PRODUCT_GROUP_APC2 <> ''
GROUP BY 1;
""")

eonn.commit()


e.execute("""
CREATE TABLE SUMM_002 AS SELECT
PRODUCT_GROUP_APC AS PRODUCT_GROUP
,SUM(M_BNM_BALANCE2) AS OSBAL
,SUM(OUTSTANDING_AFTER_EIR_UWI) AS OSBAL_IF
,SUM(OUTSTANDING_AFTER_EIR_UWI_NCIF) AS OSBAL_NCIF
,SUM(OS_AFTER_EIR_UWI_NCIF_ACCT) AS OSBAL_NCIF_ACCT
,SUM(M_BNM_BALANCE2_MIA23) AS OSBAL_MIA23
,SUM(M_BNM_BALANCE2_MIA2) AS OSBAL_MIA2
,SUM(M_BNM_BALANCE2_MIA3) AS OSBAL_MIA3
,SUM(GROSS_LOAN) AS GROSS_IF
,SUM(GROSS_LOAN_NCIF) AS GROSS_NCIF
,SUM(GROSS_LOAN_NCIF_ACCT) AS GROSS_NCIF_ACCT
, SUM(OUTSTANDING_AFTER_EIR_UWI)/SUM(M_BNM_BALANCE2) AS IF_RATIO
, SUM(OUTSTANDING_AFTER_EIR_UWI_NCIF)/SUM(M_BNM_BALANCE2) AS NCIF_RATIO
, SUM(OS_AFTER_EIR_UWI_NCIF_ACCT)/SUM(M_BNM_BALANCE2) AS NCIF_ACCT_RATIO

, SUM(GROSS_LOAN)/SUM(M_BNM_BALANCE2) AS GROSS_IF_RATIO
, SUM(GROSS_LOAN_NCIF)/SUM(M_BNM_BALANCE2) AS GROSS_NCIF_RATIO
, SUM(GROSS_LOAN_NCIF_ACCT)/SUM(M_BNM_BALANCE2) AS GROSS_NCIF_ACCT_RATIO

, SUM(M_BNM_BALANCE2_MIA23)/SUM(M_BNM_BALANCE2) AS MIA23_RATIO
, SUM(M_BNM_BALANCE2_MIA2)/SUM(M_BNM_BALANCE2) AS MIA2_RATIO
, SUM(M_BNM_BALANCE2_MIA3)/SUM(M_BNM_BALANCE2) AS MIA3_RATIO

FROM A005
WHERE PRODUCT_GROUP_APC2 = '07. OCF'
GROUP BY 1;
""")

eonn.commit()


e.execute("""
CREATE TABLE SUMM_003 AS SELECT
PRODUCT_GROUP_APC AS PRODUCT_GROUP
,M_Source_Prod_Type_Cd
,SUM(M_BNM_BALANCE2) AS OSBAL
,SUM(OUTSTANDING_AFTER_EIR_UWI) AS OSBAL_IF
,SUM(OUTSTANDING_AFTER_EIR_UWI_NCIF) AS OSBAL_NCIF
,SUM(GROSS_LOAN) AS GROSS_IF
,SUM(GROSS_LOAN_NCIF) AS GROSS_NCIF

/*,SUM(GROSS_LOAN_NCIF_ACCT) AS GROSS_NCIF_ACCT*/
/*,SUM(OS_AFTER_EIR_UWI_NCIF_ACCT) AS OSBAL_NCIF_ACCT*/

,SUM(M_BNM_BALANCE2_MIA23) AS OSBAL_MIA23
,SUM(M_BNM_BALANCE2_MIA2) AS OSBAL_MIA2
,SUM(M_BNM_BALANCE2_MIA3) AS OSBAL_MIA3
, SUM(OUTSTANDING_AFTER_EIR_UWI)/SUM(M_BNM_BALANCE2) AS IF_RATIO
, SUM(OUTSTANDING_AFTER_EIR_UWI_NCIF)/SUM(M_BNM_BALANCE2) AS NCIF_RATIO

, SUM(GROSS_LOAN)/SUM(M_BNM_BALANCE2) AS GROSS_IF_RATIO
, SUM(GROSS_LOAN_NCIF)/SUM(M_BNM_BALANCE2) AS GROSS_NCIF_RATIO

/*, SUM(GROSS_LOAN_NCIF_ACCT)/SUM(M_BNM_BALANCE2) AS GROSS_NCIF_ACCT_RATIO*/
/*, SUM(OS_AFTER_EIR_UWI_NCIF_ACCT)/SUM(M_BNM_BALANCE2) AS NCIF_ACCT_RATIO*/

, SUM(M_BNM_BALANCE2_MIA23)/SUM(M_BNM_BALANCE2) AS MIA23_RATIO
, SUM(M_BNM_BALANCE2_MIA2)/SUM(M_BNM_BALANCE2) AS MIA2_RATIO
, SUM(M_BNM_BALANCE2_MIA3)/SUM(M_BNM_BALANCE2) AS MIA3_RATIO
FROM A005
WHERE PRODUCT_GROUP_APC = '03. RSME Banking' AND M_Source_Prod_Type_Cd IN ('H74','W4')
GROUP BY 1,2;
""")

eonn.commit()

writer = pd.ExcelWriter(r'T:\MIB Risk Management\Credit Risk Analytics - Historical\01_Reporting\01_Monthly_Reporting\13_Adverse Portfolio\\'+str(date)[:4]+'\\'+str(date)+'\\working\FINAL_APC_'+str(date)+'(python).xlsx', engine='xlsxwriter')

FINAL_APC_20220238 = pd.read_sql_query("SELECT * FROM A005", eonn)
FINAL_APC_20220238.to_excel(writer, sheet_name='OVERALL', index = False)

SUMM_001 = pd.read_sql_query("SELECT * FROM SUMM_001", eonn)
SUMM_001.to_excel(writer, sheet_name='SUMM', index = False)

SUMM_002 = pd.read_sql_query("SELECT * FROM SUMM_002", eonn)
SUMM_002.to_excel(writer, sheet_name='OCF', index = False)

SUMM_003 = pd.read_sql_query("SELECT * FROM SUMM_003", eonn)
SUMM_003.to_excel(writer, sheet_name='SMEW74W4', index = False)

writer.save()


# DB SECTION¶¶
SQLite in Python (Outsource)

In [70]:
e.execute("drop table A004")
eonn.commit() 

In [17]:
eonn.close()

In [71]:
cek = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name ;", eonn)
cek

,name
